In [ ]:
import geopandas as gpd
gdf = gpd.read_file('../data/chipotle_clean.csv')
gdf = gdf.to_crs(epsg=2163)
coords = gdf[['x','y']].values


# KDE Peak Candidate
print('Finding KDE peak candidate...')
xs, ys = gdf['x'].values, gdf['y'].values
xx, yy = np.meshgrid(np.linspace(xs.min(), xs.max(), 300), np.linspace(ys.min(), ys.max(), 300))
grid = np.vstack([xx.ravel(), yy.ravel()]).T


kde = KernelDensity(bandwidth=30000).fit(np.vstack([xs, ys]).T)
Z = np.exp(kde.score_samples(grid))
argmax = Z.argmax()
peak_xy = grid[argmax]
peak_pt = gpd.GeoSeries([Point(peak_xy[0], peak_xy[1])], crs=gdf.crs).to_crs(epsg=4326)


kde_candidate = {
'method': 'kde_peak',
'lon': float(peak_pt.geometry.x.iloc[0]),
'lat': float(peak_pt.geometry.y.iloc[0])
}


# Mean k-NN Distance Candidate
print('Finding k-NN mean distance candidate...')
D = pairwise_distances(coords)
mean_k = np.sort(D, axis=1)[:, 1:11].mean(axis=1)
idx = mean_k.argmin()
pt = gdf.iloc[idx].geometry
pt_wgs = gpd.GeoSeries([pt], crs=gdf.crs).to_crs(epsg=4326).geometry.iloc[0]
knn_candidate = {
'method': 'knn_mean10',
'lon': float(pt_wgs.x),
'lat': float(pt_wgs.y),
'index': int(idx)
}


# Save candidates
candidates = {'kde_peak': kde_candidate, 'knn_mean10': knn_candidate}
with open('../outputs/candidates.json', 'w') as f:
json.dump(candidates, f, indent=2)
print('Saved candidate locations to ../outputs/candidates.json')


# Interactive Map
print('Generating interactive map...')
gdf_wgs = gdf.to_crs(epsg=4326)
center = [gdf_wgs.geometry.y.mean(), gdf_wgs.geometry.x.mean()]
m = folium.Map(location=center, zoom_start=5)


for _, row in gdf_wgs.iterrows():
folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=3, fill=True, opacity=0.6).add_to(m)


for name, loc in candidates.items():
folium.Marker(location=[loc['lat'], loc['lon']], popup=name, tooltip=name).add_to(m)


m.save('../outputs/maps/chipotle_candidates_map.html')
print('Saved interactive map to ../outputs/maps/chipotle_candidates_map.html')


# Final justification
print('Best candidate based on analysis:')
print(json.dumps(candidates, indent=2))